In [2]:
import pandas as pd

# Fairml book problem set: Data modeling of traffic stops

Data is downloaded from link in fairml book and unzipped using 7-zip. 

### Loading data

In [53]:
data_path = '..\\data\\raw\\fairml_traffic_stops\\NC_cleaned.csv'
df = pd.read_csv(data_path, low_memory = False) # low_memory False due to mixed types in columns

In [54]:
df.head()

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,search_basis,officer_id,drugs_related_stop,ethnicity,district
0,NC-2000-000001,NC,2000-01-01,00:01,NaN,NaN,NaN,Unknown,NC State Highway Patrol,M,...,Search Incident to Arrest,Incident to Arrest,False,Arrest,True,NaN,NaN,NaN,N,NaN
1,NC-2000-000002,NC,2000-01-01,00:01,NaN,NaN,NaN,Unknown,NC State Highway Patrol,M,...,NaN,NaN,False,Written Warning,False,NaN,NaN,NaN,N,NaN
2,NC-2000-000003,NC,2000-01-01,00:01,NaN,NaN,NaN,Unknown,NC State Highway Patrol,F,...,NaN,NaN,False,Written Warning,False,NaN,NaN,NaN,N,NaN
3,NC-2000-000004,NC,2000-01-01,00:02,NaN,NaN,NaN,Unknown,NC State Highway Patrol,F,...,NaN,NaN,False,Citation,False,NaN,NaN,NaN,N,NaN
4,NC-2000-000005,NC,2000-01-01,00:02,NaN,NaN,NaN,Unknown,NC State Highway Patrol,F,...,NaN,NaN,False,No Action,False,NaN,NaN,NaN,N,NaN


In [55]:
df.dtypes

id                        object
state                     object
stop_date                 object
stop_time                 object
location_raw              object
county_name               object
county_fips              float64
fine_grained_location     object
police_department         object
driver_gender             object
driver_age_raw           float64
driver_age               float64
driver_race_raw           object
driver_race               object
violation_raw             object
violation                 object
search_conducted            bool
search_type_raw           object
search_type               object
contraband_found            bool
stop_outcome              object
is_arrested                 bool
search_basis              object
officer_id                object
drugs_related_stop        object
ethnicity                 object
district                  object
dtype: object

In [56]:
df.shape

(9558084, 27)

In [57]:
df.id.is_unique # All rows are unique

True

## Part A 
### Counting stop rates and population sizes

First the data is cleaned to contain only the relevant categories and age is aggregated into the age groups

In [74]:
# Fixing race
race_groups = ['Asian', 'Black', 'Hispanic', 'White']
df_clean = df[df.driver_race.isin(race_groups)].copy()
df_clean.shape

(9310376, 27)

In [75]:
# Exists entry without age - this entry is filtered out
print(df_clean.driver_age.unique()) 
df_clean = df_clean[df_clean.driver_age.notnull()].copy()
max_age = max(df_clean.driver_age)
df_clean.shape

[35. 20. 26. 48. 18. 25. 30. 40. 28. 21. 22. 31. 23. 27. 50. 44. 17. 19.
 60. 33. 45. 37. 49. 24. 54. 41. 63. 29. 67. 32. 16. 57. 36. 46. 86. 43.
 42. 77. 59. 34. 38. 51. 66. 65. 79. 39. 52. 64. 58. 53. 85. 69. 47. 72.
 71. 56. 15. 55. 74. 62. 61. 68. 76. nan 81. 70. 73. 78. 75. 84. 83. 80.
 92. 90. 82. 97. 87. 91. 89. 94. 93. 96. 88. 95. 99. 98.]


(9308215, 27)

In [77]:
# Fixing age
age_groups = [15, 20, 30, 40, 50, max_age+1]
df_clean['driver_age_group'] = pd.cut(
    df_clean.driver_age, 
    bins = age_groups, 
    right = False)


(9308215, 28)

In [15]:
# Gender is as it should be
df_clean.driver_gender.unique()

array(['F', 'M'], dtype=object)

In [78]:
# District
# Removing all with unknown districts
df_clean = df_clean[df_clean.district.notnull()].copy()

(8979701, 28)

In [79]:
# Year
df_clean['stop_year'] = df_clean.stop_date.str.slice(0,4).astype(int)
df_clean = df_clean[(df_clean.stop_year > 2009) & (df_clean.stop_year < 2016)]

(4312841, 29)

In [88]:
# Making final table
group_cols = ['driver_race', 'driver_age_group', 'driver_gender', 'district', 'stop_year']
count_table = df_clean.groupby(group_cols).size().reset_index(name = 'n_stops')
count_table.driver_age_group = count_table.driver_age_group.astype("str")

In [89]:
count_table.head(10)

,driver_race,driver_age_group,driver_gender,district,stop_year,n_stops
0,Asian,"[15.0, 20.0)",F,A1,2010,1
1,Asian,"[15.0, 20.0)",F,A1,2011,1
2,Asian,"[15.0, 20.0)",F,A1,2012,1
3,Asian,"[15.0, 20.0)",F,A1,2013,0
4,Asian,"[15.0, 20.0)",F,A1,2014,0
5,Asian,"[15.0, 20.0)",F,A1,2015,2
6,Asian,"[15.0, 20.0)",F,A2,2010,0
7,Asian,"[15.0, 20.0)",F,A2,2011,0
8,Asian,"[15.0, 20.0)",F,A2,2012,0
9,Asian,"[15.0, 20.0)",F,A2,2013,2


In [90]:
# Yay!
count_table[(count_table.driver_race == 'Hispanic') & (count_table.driver_age_group == '[30.0, 40.0)') & (count_table.driver_gender == 'F')& (count_table.district == 'B5') & (count_table.stop_year == 2012)] 

,driver_race,driver_age_group,driver_gender,district,stop_year,n_stops
8288,Hispanic,"[30.0, 40.0)",F,B5,2012,76


In [94]:
# Yay - must be correct now :D!
count_table[
    (count_table.driver_race == 'White') \
    & (count_table.driver_age_group == '[40.0, 50.0)') \
    & (count_table.driver_gender == 'F') \
    & (count_table.district == 'C8') \
    & (count_table.stop_year == 2011)
] 

,driver_race,driver_age_group,driver_gender,district,stop_year,n_stops
12457,White,"[40.0, 50.0)",F,C8,2011,213
